In [1]:
pos(p) = p + 1;
ref(a,p) = a[pos(a[pos(p)])];
@enum Op plus=1 mul=2 stor=3 out=4 jnz=5 jez=6 lt=7 eq=8 off=9 halt=99
@enum Mode posi=0 imm=1 rel=2

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end

getOps(str::String) = getOps(parse(Int,str))
function getOps(num)  
    
        op = Op(num % 100)
        mode1 = Mode((num ÷ 100)  % 10)
        mode2 = Mode((num ÷ 1000)  % 10)
        mode3 = Mode((num ÷ 10000)  % 10)
        
        Operation(op,mode1,mode2,mode3)
end

function op1(operation,p,offset,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param =  prog[pos(p+1)]
        off_p = offset + param
        p1 = prog[pos(off_p)]
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
        param =  prog[pos(p+2)]
        off_p = offset + param
        p2 = prog[pos(off_p)]
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
 
    if operation.mode3 == posi        
        p3 = prog[pos(p+3)]
    elseif operation.mode3 == rel        
        param = prog[pos(p+3)]
        off_p = offset + param
        p3 = off_p
    elseif operation.mode3 == imm     
        p3 = prog[pos(p+3)]
    end     
    prog[pos(p3)] = p1 + p2
    p + 4
end

function op2(operation,p,offset,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        off_p = offset + param
        p1 = prog[pos(off_p)]
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
        param = prog[pos(p+2)]
        off_p = offset + param
        p2 = prog[pos(off_p)]
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if operation.mode3 == posi        
        p3 = prog[pos(p+3)]
    elseif operation.mode3 == rel        
        param = prog[pos(p+3)]
        off_p = offset + param
        p3 = off_p  
    elseif operation.mode3 == imm     
        p3 = prog[pos(p+3)]
    end
    prog[pos(p3)] = p1 * p2
    p + 4
end

function op3(operation,p,offset,prog,input) 
    if operation.mode1 == posi
        p1 = prog[pos(p+1)]
        prog[pos(p1)] = input      
    elseif operation.mode1 == rel
        param = prog[pos(p+1)] # get parameter
        off_p = offset + param # apply offset
        prog[pos(off_p)] = input # put input into position in memory
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
        prog[pos(p1)] = input 
    end     
    p + 2
end

function op4(operation,p,offset,prog,output) #out  
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)] # get the parameter
        off_p = offset + param # get the position with its offset
        p1 = prog[pos(off_p)] # read the memory address at offset + parameter
        #error("implement")  
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    push!(output,p1)
    (p + 2,output)
end



# Opcode 5 is jump-if-true: 
# if the first parameter is non-zero, 
# it sets the instruction pointer to the value from the second parameter. 
# Otherwise, it does nothing.

function op5(operation,p,offset,prog)   
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        p_off = offset + param
        p1 = prog[pos(p_off)] 
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2) 
    elseif operation.mode2 == rel
        param =  prog[pos(p+2)]
        p_off = offset + param
        p2 = prog[pos(p_off)] 
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    
    if p1 != 0
        return p2
    else
        return p + 3
    end
end

# Opcode 6 is jump-if-false: 
# if the first parameter is zero, 
# it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

function op6(operation,p,offset,prog)      
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        p_off = offset + param
        p1 = prog[pos(p_off)]                
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
        param =  prog[pos(p+2)]
        p_off = offset + param
        p2 = prog[pos(p_off)]                     
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
  
    if p1 == 0
        return p2
    else
        return p + 3
    end
end

# Opcode 7 is less than: 
# if the first parameter is less than the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op7(operation,p,offset,prog)     
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        off_p = offset + param
        p1 = prog[pos(off_p)]
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
       param =  prog[pos(p+2)]
        off_p = offset + param
        p2 = prog[pos(off_p)]
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    result = 0
    if p1 < p2
        result = 1
    end
    
    if operation.mode3 == posi       
        p3 = prog[pos(p+3)]
        #p3 = ref(prog,p+1)
    elseif operation.mode3 == rel        
        param = prog[pos(p+3)]
        p3 = offset + param  
    elseif operation.mode3 == imm        
        p3 = prog[pos(p+3)]
    end  
    
    prog[pos(p3)] = result

    p + 4
end

# Opcode 8 is equals: 
# if the first parameter is equal to the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op8(operation,p,offset,prog) 

    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        off_p = offset + param
        p1 = prog[pos(off_p)]
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi         
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel         
       param = prog[pos(p+2)] # get the parameter
       off_p = offset + param # apply the offset
       p2 = prog[pos(off_p)] # get the value stored at this position 
    elseif operation.mode2 == imm      
        p2 = prog[pos(p+2)]
    end  
    
    result = 0
    if p1 == p2
        result = 1
    end
        
    if operation.mode3 == posi        
        p3 = prog[pos(p+3)]
    elseif operation.mode3 == rel
        #error("implement")
        param = prog[pos(p+3)]
        p3 = offset + param
        #p3 = prog[pos(off_p)]
    elseif operation.mode3 == imm        
        p3 = prog[pos(p+3)]
    end    
    prog[pos(p3)] = result

    p + 4
end

#Opcode 9 adjusts the relative base by the value of its only parameter. 
#The relative base increases  
#    (or decreases, if the value is negative) 
#     by the value of the parameter.
#For example, if the relative base is 2000, 
#    then after the instruction 109,19, 
#    the relative base would be 2019. 
#If the next instruction were 204,-34, 
#    then the value at address 1985 would be output.

function op9(operation,p,offset,prog) 

    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        p1 = prog[pos(offset+param)]       
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  

    offset = offset + p1

    (p + 2,offset)
end





function execute(op::Operation, p, offset, prog, input,output)

    if op.op == plus
        return (op1(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == mul
        return (op2(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == stor
        if length(input) == 0 
            return (p,input,output,true)
        end
        next = input[1]
        input=input[2:end]
        return (op3(op,p,offset,prog,next),input,output,offset,false)
    end
    if op.op == out
        (p1,output) = op4(op,p,offset,prog,output)
        return (p1,input,output,offset,false)
    end
    if op.op == jnz
        return (op5(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == jez
        return (op6(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == lt
        return (op7(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == eq
        return (op8(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == off
        (p,offset) = op9(op,p,offset,prog)
        return (p,input,output,offset,false)
    end 
    
    error("panic executing unknown op")
end

function execute(prog,p,offset,input,output)
    push!(prog,zeros(Int,100000)...)
    op = getOps(prog[pos(p)])
    while op.op != halt
        (p,input,output,offset,halt) = execute(op,p,offset,prog,input,output)
        if halt
            return (prog,p,output,:pause)
        end
        op = getOps(prog[pos(p)])
    end 
    (prog,p,output,:over)
end

function execute(prog)
  execute(prog,0,0,[],[])
end

(prog,p,output,state) = execute([1,9,10,3,2,3,11,0,99,30,40,50]) # == [3500,9,10,70,2,3,11,0,99,30,40,50])
println(prog[1:12])
#println(execute([1,0,0,0,99]) == [2,0,0,0,99])
#println(execute([2,3,0,3,99]) == [2,3,0,6,99])
#println(execute([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
#println(execute([1,1,1,4,99,5,6,0,99]) == [ 30,1,1,4,2,5,6,0,99])

# The above example program uses an input instruction to ask for a single number. 
# The program will then 
# output 999 if the input value is below 8,
# output 1000 if the input value is equal to 8, 
# or output 1001 if the input value is greater than 8.

(prog,p,output,state) =  (execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,0,[-8],[]))
println("\n\ntest $output\n\n")
(prog,p,output,state) =  (execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,0,[8],[]))
println("\n\ntest $output\n\n")
(prog,p,output,state) =  execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,0,[16],[]);

println("\n\ntest $output\n\n")

[3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]


test Any[999]




test Any[1000]




test Any[1001]




```
--- Day 11: Space Police ---

On the way to Jupiter, you're pulled over by the Space Police.

"Attention, unmarked spacecraft! You are in violation of Space Law! All spacecraft must have a clearly visible registration identifier! You have 24 hours to comply or be sent to Space Jail!"

Not wanting to be sent to Space Jail, you radio back to the Elves on Earth for help. Although it takes almost three hours for their reply signal to reach you, they send instructions for how to power up the emergency hull painting robot and even provide a small Intcode program (your puzzle input) that will cause it to paint your ship appropriately.

There's just one problem: you don't have an emergency hull painting robot.

You'll need to build a new emergency hull painting robot. The robot needs to be able to move around on the grid of square panels on the side of your ship, detect the color of its current panel, and paint its current panel black or white. (All of the panels are currently black.)

The Intcode program will serve as the brain of the robot. The program uses input instructions to access the robot's camera: provide 0 if the robot is over a black panel or 1 if the robot is over a white panel. Then, the program will output two values:

First, it will output a value indicating the color to paint the panel the robot is over: 0 means to paint the panel black, and 1 means to paint the panel white.
Second, it will output a value indicating the direction the robot should turn: 0 means it should turn left 90 degrees, and 1 means it should turn right 90 degrees.
After the robot turns, it should always move forward exactly one panel. The robot starts facing up.

The robot will continue running for a while like this and halt when it is finished drawing. Do not restart the Intcode computer inside the robot during this process.

For example, suppose the robot is about to start running. Drawing black panels as ., white panels as #, and the robot pointing the direction it is facing (< ^ > v), the initial state and region near the robot looks like this:

.....
.....
..^..
.....
.....
The panel under the robot (not visible here because a ^ is shown instead) is also black, and so any input instructions at this point should be provided 0. Suppose the robot eventually outputs 1 (paint white) and then 0 (turn left). After taking these actions and moving forward one panel, the region now looks like this:

.....
.....
.<#..
.....
.....
Input instructions should still be provided 0. Next, the robot might output 0 (paint black) and then 0 (turn left):

.....
.....
..#..
.v...
.....
After more outputs (1,0, 1,0):

.....
.....
..^..
.##..
.....
The robot is now back where it started, but because it is now on a white panel, input instructions should be provided 1. After several more outputs (0,1, 1,0, 1,0), the area looks like this:

.....
..<#.
...#.
.##..
.....
Before you deploy the robot, you should probably have an estimate of the area it will cover: specifically, you need to know the number of panels it paints at least once, regardless of color. In the example above, the robot painted 6 panels at least once. (It painted its starting panel twice, but that panel is still only counted once; it also never painted the panel it ended on.)

Build a new emergency hull painting robot and run the Intcode program on it. How many panels does it paint at least once?

Your puzzle answer was 2219.
```

In [160]:
#override execute method for the day's problem 
direction = :up
brain_mode = :paint
xy = (0,0)
grid=Dict{Tuple{Int,Int},Int}()
minx = 10000
maxx = -100000
miny = 10000
maxy = -10000
#grid[(0,0)] = 0
function execute(op::Operation, p, offset, prog, input,output)
    global brain_mode
    global direction
    global xy
    global grid
    global minx
    global miny 
    global maxy 
    global maxx

    if op.op == plus
        return (op1(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == mul
        return (op2(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == stor
        if length(input) == 0 
            return (p,input,output,true)
        end
        next = input[1]
        input=input[2:end]
        return (op3(op,p,offset,prog,next),input,output,offset,false)
    end
    if op.op == out
        (p1,output) = op4(op,p,offset,prog,output)
        #println(output)
        if length(output) == 2
            # First, it will output a value indicating the color to paint the panel the robot is over: 
            #    * 0 means to paint the panel black, and 
            #    * 1 means to paint the panel white. 
            grid[xy] = output[1] # paint the ground the correct colour
            # Second, it will output a value indicating the direction the robot should turn: 
            #   * 0 means it should turn left 90 degrees, and 
            #   * 1 means it should turn right 90 degrees.         
            direction = change_dir(direction,output[2])
            # After the robot turns, it should always move forward exactly one panel. The robot starts facing up.        
            (dx,dy) = robot_move(xy,direction)
            (x,y) = xy
            x = x + dx
            y = y + dy
            xy = (x,y)
            minx = x < minx ? x : minx
            miny = y < miny ? y : miny
            maxx = x > maxx ? x : maxx
            maxy = y > maxx ? y : maxy
            output = []
            # Read panel colour
            push!(input,get(grid,xy,0))
        end 
        return (p1,input,output,offset,false)
    end
    if op.op == jnz
        return (op5(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == jez
        return (op6(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == lt
        return (op7(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == eq
        return (op8(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == off
        (p,offset) = op9(op,p,offset,prog)
        return (p,input,output,offset,false)
    end 
    
    error("panic executing unknown op")
end

execute (generic function with 3 methods)

In [161]:
function robot_move(xy,direction)
    if direction == :up     return (0,-1) end
    if direction == :down   return (0,1) end
    if direction == :left   return (-1,0) end
    if direction == :right  return (+1,0) end
end

function change_dir(direction, change)
    left = 0
    right = 1
    if direction == :up    && change == left  return :left end
    if direction == :down  && change == left  return :right end
    if direction == :left  && change == left  return :down end
    if direction == :right && change == left  return :up end   
    
    if direction == :up    && change == right  return :right end
    if direction == :down  && change == right  return :left end
    if direction == :left  && change == right  return :up end
    if direction == :right && change == right  return :down end
end

change_dir (generic function with 1 method)

In [162]:
using DelimitedFiles
prog = readdlm("input.txt", '\t', Int, ',')[:,1]
(prog,p,output,state) =  (execute(prog,0,0,[0],[]))
length(grid)

2219

```
--- Part Two ---

You're not sure what it's trying to paint, but it's definitely not a registration identifier. The Space Police are getting impatient.

Checking your external ship cameras again, you notice a white panel marked "emergency hull painting robot starting panel". The rest of the panels are still black, but it looks like the robot was expecting to start on a white panel, not a black one.

Based on the Space Law Space Brochure that the Space Police attached to one of your windows, a valid registration identifier is always eight capital letters. After starting the robot on a single white panel instead, what registration identifier does it paint on your hull?

Your puzzle answer was HAFULAPE.
```

In [184]:
direction = :up
brain_mode = :paint
xy = (0,0)
grid=Dict{Tuple{Int,Int},Int}()
grid[xy] = 1
prog = readdlm("input.txt", '\t', Int, ',')[:,1]
(prog,p,output,state) =  (execute(prog,0,0,[1],[]))
for i ∈ maxy-10:maxy-1
    for j ∈ minx+5:maxx-10
        color = get(grid,(j,i),0)
        if color == 0
            print("😈")
        else
            print("😜")
        end
    end
    println()
end

😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈
😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈
😈😈😜😈😈😜😈😈😜😜😈😈😜😜😜😜😈😜😈😈😜😈😜😈😈😈😈😈😜😜😈😈😜😜😜😈😈😜😜😜😜😈😈
😈😈😜😈😈😜😈😜😈😈😜😈😜😈😈😈😈😜😈😈😜😈😜😈😈😈😈😜😈😈😜😈😜😈😈😜😈😜😈😈😈😈😈
😈😈😜😜😜😜😈😜😈😈😜😈😜😜😜😈😈😜😈😈😜😈😜😈😈😈😈😜😈😈😜😈😜😈😈😜😈😜😜😜😈😈😈
😈😈😜😈😈😜😈😜😜😜😜😈😜😈😈😈😈😜😈😈😜😈😜😈😈😈😈😜😜😜😜😈😜😜😜😈😈😜😈😈😈😈😈
😈😈😜😈😈😜😈😜😈😈😜😈😜😈😈😈😈😜😈😈😜😈😜😈😈😈😈😜😈😈😜😈😜😈😈😈😈😜😈😈😈😈😈
😈😈😜😈😈😜😈😜😈😈😜😈😜😈😈😈😈😈😜😜😈😈😜😜😜😜😈😜😈😈😜😈😜😈😈😈😈😜😜😜😜😈😈
😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈
😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈😈
